In [1]:
import urllib3
from bs4 import BeautifulSoup
import requests
import json
import re
import datetime
import pandas as pd
import numpy as np
from tqdm import tqdm
import os
import time

In [2]:
os.chdir("../")

In [3]:
base_url = "https://www.imdb.com/"

In [4]:
df = pd.read_csv('./cache/actors.csv')

In [6]:
df.head(1)

,name,url
0,Tim Robbins,/name/nm0000209


In [11]:
def get_soup(path_url):
    try:
        full_url = base_url + path_url
        headers = {'User-Agent': 'Thunder Client (https://www.thunderclient.com)'}

        time.sleep(1)
        res = requests.get(full_url, headers)
        soup = BeautifulSoup(res.text, 'html.parser')
        return soup
    except Exception as e:
        raise e

In [19]:
url = '/name/nm1500155'
job_type = 'actor'

if job_type not in ('director','actor','writer'):
    raise ValueError("Invalid input for job_type")

url = url.replace("/name/", "")
full_url = f'filmosearch/?explore=title_type&role={url}&ref_=filmo_ref_job_typ&sort=num_votes,desc&mode=detail&page=1&job_type={job_type}&title_type=movie'
soup = get_soup(full_url)
print(full_url)

filmosearch/?explore=title_type&role=nm1500155&ref_=filmo_ref_job_typ&sort=num_votes,desc&mode=detail&page=1&job_type=actor&title_type=movie


In [60]:
filmo = []
for movie in soup.select("div.lister-item-content"):
    row = dict()
    try:
        name = movie.select_one("a").text.strip()
        row['name'] = name
    except AttributeError:
        row['name'] = None

    try:
        year = movie.select_one("span.lister-item-year.text-muted.unbold").text.strip()
        row['year'] = year
    except AttributeError:
        row['year'] = None

    try:
        cert = movie.select_one("span.certificate").text.strip()
        row['cert'] = cert
    except AttributeError:
        row['cert'] = None

    try:
        runtime = movie.select_one("span.runtime").text.strip()
        row['runtime'] = runtime
    except AttributeError:
        row['runtime'] = None

    try:
        genres = movie.select_one("span.genre").text.strip()
        row['genres'] = genres
    except AttributeError:
        row['genres'] = None

    try:
        rating = movie.select_one("div.inline-block.ratings-imdb-rating > strong").text.strip()
        row['rating'] = rating
    except AttributeError:
        row['rating'] = None
    try:
        label = movie.select("p.text-muted.text-small")

        for l in label:
            if 'Director' in l.text:
                small_text = [re.sub("(\\n)", "", t).strip().split(":")[1].split(",") for t in l.text.split("| ")]
                break

        row['director'] = small_text[0]
        row['stars'] = small_text[1]

        # print(small_text)
    except AttributeError:
        row['director'] = None
        row['stars'] = None

    try:
        vote_count = [nv.text for nv in movie.find_all("span", {"name": "nv"})][0]
        row['vote_count'] = vote_count
    except:
        row['vote_count'] = None

    try:
        revenue = [nv.text for nv in movie.find_all("span", {"name": "nv"})][1]
        row['revenue'] = revenue
    except:
        row['revenue'] = None

    filmo.append(row)
    # print(row)
    # break

df = pd.DataFrame(filmo)

df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 30 entries, 0 to 29
Data columns (total 10 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   name        30 non-null     object
 1   year        30 non-null     object
 2   cert        25 non-null     object
 3   runtime     28 non-null     object
 4   genres      30 non-null     object
 5   rating      28 non-null     object
 6   director    30 non-null     object
 7   stars       30 non-null     object
 8   vote_count  28 non-null     object
 9   revenue     20 non-null     object
dtypes: object(10)
memory usage: 2.5+ KB


In [59]:
df

,name,year,cert,runtime,genres,rating,director,stars,vote_count,revenue
0,Harry Potter và Chiếc Cốc Lửa,(2005),PG-13,157 min,"Adventure, Family, Fantasy",7.7,[Mike Newell],"[Daniel Radcliffe, Emma Watson, Rupert Grint...","607,075",$290.01M
1,Tenet,(2020),C16,150 min,"Action, Adventure, Sci-Fi",7.3,[Christopher Nolan],"[John David Washington, Robert Pattinson, El...","466,816",$58.46M
2,Chạng Vạng,(I) (2008),PG-13,122 min,"Drama, Fantasy, Romance",5.3,[Catherine Hardwicke],"[Kristen Stewart, Robert Pattinson, Billy Bu...","454,033",$191.47M
3,Batman: Vạch Trần Sự Thật,(2022),C16,176 min,"Action, Crime, Drama",8.0,[Matt Reeves],"[Robert Pattinson, Zoë Kravitz, Jeffrey Wrig...","448,980",None
4,Trăng Non,(2009),PG-13,130 min,"Adventure, Drama, Fantasy",4.7,[Chris Weitz],"[Kristen Stewart, Robert Pattinson, Taylor L...","280,588",$296.62M
5,Chạng Vạng: Hừng Đông - Phần 2,(2012),C16,115 min,"Adventure, Drama, Fantasy",5.5,[Bill Condon],"[Kristen Stewart, Robert Pattinson, Taylor L...","245,326",$292.30M
6,The Twilight Saga: Nhât thuc,(2010),PG-13,124 min,"Action, Adventure, Drama",5.0,[David Slade],"[Kristen Stewart, Robert Pattinson, Taylor L...","242,403",$300.53M
7,Chạng Vạng: Hừng Đông - Phần 1,(2011),PG-13,117 min,"Adventure, Drama, Fantasy",4.9,[Bill Condon],"[Kristen Stewart, Robert Pattinson, Taylor L...","237,235",$281.29M
8,Ánh Sáng Tăm Tối,(I) (2019),R,109 min,"Drama, Fantasy, History",7.4,[Robert Eggers],"[Robert Pattinson, Willem Dafoe, Valeriia Ka...","200,212",$0.43M
9,Nhớ Đến Anh,(I) (2010),PG-13,113 min,"Drama, Romance",7.1,[Allen Coulter],"[Robert Pattinson, Emilie de Ravin, Caitlyn ...","145,661",$19.07M
